In [2]:
import sys
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
import sys
import h5py
from sklearn import metrics
import seaborn as sns
import ncempy.io.dm as dm
import ncempy.io.mrc as mrc
sys.path.append('pyNanoFind/')
import realDataProcess as rdp
import segmentationAnalysis as sega
from keras.models import Model
from keras.layers import *
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.losses import binary_crossentropy
from keras.models import load_model
import pandas as pd
from matplotlib.patches import Rectangle
import scipy.fftpack as ftp
from scipy.signal import medfilt2d
from scipy import signal
import CNNviz as cviz

Using TensorFlow backend.


In [ ]:
# load data - data is available via Zenodo: https://zenodo.org/record/3755011#.Xpo0mlNKgUE
combined = h5py.File('Bal_MedFilt_AuCdSeDots_20190726_v2.h5')['images'][:,:,:,:]
combinedMaps = h5py.File('Bal_MedFilt_AuCdSeDots_20190726_maps_v2.h5')['maps'][:,:,:,:]

In [ ]:
# load model weights for combined network
model_combined = cviz.get_unet((512,512,1),2)
model_combined.load_weights('trained_models/unet32_AuCdSeDots_09_final.h5')

In [ ]:
# predict segmentation maps on training sets
pred_train_combined = model_combined.predict(combined[:129])

In [ ]:
# calculate threshold on the training set
acc_combined, thresh_combined = sega.find_thresh(combinedMaps[:129],pred_train_combined,thresh=0.8,step=0.02)

In [ ]:
# predict on the test set
pred_test = model_combined.predict(combinedMaps[129:])

In [ ]:
# make segmentation map binary
binary_pred_test = sega.binarize_map(pred_test[:,:,:,1],thresh_combined)

In [ ]:
# view binary segmentation map overlayed on original image
rdp.imm(combined[129:],binary_pred_test,0)

In [ ]:
# calculate confusion matrix
cfm = metrics.confusion_matrix(combinedMaps[129:].flatten(),binary_pred_test.flatten())
sega.plot_confusion_matrix(cfm,['background','particle'],'jet',True)